In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 기본 directory 설정
import os
os.chdir('/content/drive/MyDrive/Monthly_Workout')

In [3]:
# 모듈 불러오기
import random
import pandas as pd
import numpy as np
from tqdm import tqdm

In [4]:
# 데이터 불러오기
path = './' # 기본 directory 경로에 추가 할 경로

train = pd.read_csv(path + 'train_features.csv')
train_labels = pd.read_csv(path + 'train_labels.csv')
test = pd.read_csv(path + 'test_features.csv')
submission = pd.read_csv(path + 'sample_submission.csv')

In [5]:
act_list=train.iloc[:,2:].columns
acc_list=['acc_x','acc_y','acc_z']
gy_list=['gy_x','gy_y','gy_z']
act_list

Index(['acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z'], dtype='object')

In [6]:
from sklearn.preprocessing import StandardScaler
# Scaling
scaler=StandardScaler()
train[act_list]=scaler.fit_transform(train[act_list])
test[act_list]=scaler.transform(test[act_list])
train

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,0,1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,0,2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,0,3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,0,4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.367432,-0.845648,0.503707,-0.461266,-1.413128,-1.092639
1874996,3124,596,-0.326315,-0.845833,0.578875,-0.474109,-1.382884,-1.097377
1874997,3124,597,-0.300794,-0.859728,0.652722,-0.436694,-1.341672,-1.136169
1874998,3124,598,-0.253120,-0.886873,0.673002,-0.367051,-1.346630,-1.148831


In [7]:
# acc 데이터와 gy 데이터로 분할
def sensor_split(data):
    X_acc = []
    X_gy = []

    for i in tqdm(data['id'].unique()):
        temp_acc = np.array(data[data['id'] == i].loc[:,acc_list])
        temp_gy = np.array(data[data['id'] == i].loc[:,gy_list])
        X_acc.append(temp_acc)
        X_gy.append(temp_gy)
      
    X_acc = np.array(X_acc).reshape(-1,600,3)
    X_gy = np.array(X_gy).reshape(-1,600,3)

    return X_acc, X_gy

In [8]:
# acc, gy 데이터 분할
X_train_mod=pd.merge(train,train_labels,how='left',on='id')
X_train_acc, X_train_gy= sensor_split(X_train_mod)

X_train_acc.shape, X_train_gy.shape

100%|██████████| 3125/3125 [00:18<00:00, 172.72it/s]


((3125, 600, 3), (3125, 600, 3))

In [9]:
# 데이터 증강

def Permutation(X, nPerm=4, minSegLength=10):
    X_new = np.zeros(X.shape)
    idx = np.random.permutation(nPerm)
    bWhile = True
    while bWhile == True:
        segs = np.zeros(nPerm+1, dtype=int)
        segs[1:-1] = np.sort(np.random.randint(minSegLength, X.shape[0]-minSegLength, nPerm-1))
        segs[-1] = X.shape[0]
        if np.min(segs[1:]-segs[0:-1]) > minSegLength:
            bWhile = False
    pp = 0
    for ii in range(nPerm):
        x_temp = X[segs[idx[ii]]:segs[idx[ii]+1],:]
        X_new[pp:pp+len(x_temp),:] = x_temp
        pp += len(x_temp)
    return (X_new)

def aug(data, uid, shift):
    shift_data = np.roll(data[uid], shift, axis=0)
    return shift_data
def Rolling(data):
    aug_data=[]
    for i in range(data.shape[0]):
        temp=list((aug(data,i,int(random.random()*600))))
        aug_data.append(temp)
    return np.array(aug_data)

In [10]:
# 데이터 증강 (반복하고 싶은 만큼 조정)
X_train_acc_temp = X_train_acc.copy()
X_train_gy_temp = X_train_gy.copy()

y_train = train_labels['label']
y_train_total = np.append(y_train, y_train, axis=0)

rep = 5
for i in range(rep):
    X_train_acc_roll = Rolling(X_train_acc_temp)
    X_train_acc_rp = Permutation(Rolling(X_train_acc_temp)) # rolling + permutation

    X_train_gy_roll = Rolling(X_train_gy_temp)
    X_train_gy_rp = Permutation(Rolling(X_train_gy_temp)) # rolling + permutation

    # 증강시킨 데이터 원래 데이터에 추가
    X_train_acc = np.append(X_train_acc, X_train_acc_roll, axis=0)
    X_train_acc = np.append(X_train_acc, X_train_acc_rp, axis=0)

    X_train_gy = np.append(X_train_gy, X_train_gy_roll, axis=0)
    X_train_gy = np.append(X_train_gy, X_train_gy_rp, axis=0)

    y_train_total = np.append(y_train_total, y_train, axis=0)
    if i != (rep-1): # 마지막 한 번 제외
        y_train_total = np.append(y_train_total, y_train, axis=0)
        
X_train_acc.shape, X_train_gy.shape, y_train_total.shape

((34375, 600, 3), (34375, 600, 3), (34375,))

In [11]:
# np array 형태를 dataframe 으로 변환
acc = [e for sl in X_train_acc for e in sl]

df_report = np.stack(acc, axis = 0)
df_acc = pd.DataFrame(df_report, columns= ['acc_x', 'acc_y', 'acc_z']) 

df_acc

,acc_x,acc_y,acc_z
0,2.307314,-0.029939,-0.432104
1,2.421086,-0.063321,-0.504058
2,2.444664,-0.056749,-0.654199
3,2.428612,-0.116782,-0.573407
4,2.439475,-0.044220,-0.588886
...,...,...,...
20624995,-0.840162,0.249148,-0.014579
20624996,-0.907620,0.225072,-0.044249
20624997,-0.878619,0.186470,-0.029572
20624998,-0.818231,0.203014,-0.004935


In [12]:
# gy
gy = [e for sl in X_train_gy for e in sl]

df_report = np.stack(gy, axis = 0)
df_gy = pd.DataFrame(df_report, columns= ['gy_x', 'gy_y', 'gy_z']) 

df_gy

,gy_x,gy_y,gy_z
0,0.011776,-0.410859,-0.461007
1,0.026484,-0.528052,-0.365459
2,-0.037962,-0.596041,-0.366767
3,0.066099,-0.725311,-0.401232
4,0.091969,-0.784094,-0.408413
...,...,...,...
20624995,-0.285672,0.071019,-0.173637
20624996,-0.436427,0.091745,-0.087905
20624997,-0.336794,0.076047,-0.066560
20624998,-0.231834,0.014088,-0.141685


In [13]:
# acc, gy 데이터프레임 병합
df_aug_result = pd.concat([df_acc, df_gy], axis = 1)

train = df_aug_result
train

,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413
...,...,...,...,...,...,...
20624995,-0.840162,0.249148,-0.014579,-0.285672,0.071019,-0.173637
20624996,-0.907620,0.225072,-0.044249,-0.436427,0.091745,-0.087905
20624997,-0.878619,0.186470,-0.029572,-0.336794,0.076047,-0.066560
20624998,-0.818231,0.203014,-0.004935,-0.231834,0.014088,-0.141685


In [14]:
# 속도
train['acc_t'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2) ** (1/3)
test['acc_t'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2) ** (1/3)

train['gy_t'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2) ** (1/3)
test['gy_t'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2) ** (1/3)

# Signal 극대화 (peak 캐치 유용)
train['acc_mag'] = (train['acc_x'] ** 2) + (train['acc_y'] ** 2) + (train['acc_z'] ** 2)
test['acc_mag'] = (test['acc_x'] ** 2) + (test['acc_y'] ** 2) + (test['acc_z'] ** 2)

train['gy_mag'] = (train['gy_x'] ** 2) + (train['gy_y'] ** 2) + (train['gy_z'] ** 2)
test['gy_mag'] = (test['gy_x'] ** 2) + (test['gy_y'] ** 2) + (test['gy_z'] ** 2)

In [15]:
# vector
train['acc_vector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2)+(train['acc_z'] ** 2))
test['acc_vector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2)+(test['acc_z'] ** 2))

train['gy_vector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2)+(train['gy_z'] ** 2))
test['gy_vector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2)+(test['gy_z'] ** 2))

train['acc_YZvector'] = np.sqrt((train['acc_y'] ** 2) +(train['acc_z'] ** 2))
test['acc_YZvector'] = np.sqrt((test['acc_y'] ** 2) +(test['acc_z'] ** 2))

train['gy_YZvector'] = np.sqrt((train['gy_y'] ** 2) +(train['gy_z'] ** 2))
test['gy_YZvector'] = np.sqrt((test['gy_y'] ** 2) +(test['gy_z'] ** 2))

train['acc_XYvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_y'] ** 2))
test['acc_XYvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_y'] ** 2))

train['gy_XYvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_y'] ** 2))
test['gy_XYvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_y'] ** 2))

train['acc_XZvector'] = np.sqrt((train['acc_x'] ** 2) +(train['acc_z'] ** 2))
test['acc_XZvector'] = np.sqrt((test['acc_x'] ** 2) +(test['acc_z'] ** 2))

train['gy_XZvector'] = np.sqrt((train['gy_x'] ** 2) +(train['gy_z'] ** 2))
test['gy_XZvector'] = np.sqrt((test['gy_x'] ** 2) +(test['gy_z'] ** 2))

# 자이로스코프 무게중심
train['gy_Centerofgravity'] = (train['gy_x']+train['gy_y']+train['gy_z'])/3
test['gy_Centerofgravity'] = (test['gy_x']+test['gy_y']+test['gy_z'])/3

In [16]:
from math import pi

# roll & pitch
train['roll'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2))
test['roll'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2))

train['pitch'] = np.arctan(train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2))
test['pitch'] = np.arctan(test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2))

train['math_roll'] = np.arctan(- train['acc_x']/np.sqrt(train['acc_y'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_roll'] = np.arctan(- test['acc_x']/np.sqrt(test['acc_y'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['math_pitch'] = np.arctan(train['acc_y']/np.sqrt(train['acc_x'] ** 2 + train['acc_z'] ** 2)) * (180/pi)
test['math_pitch'] = np.arctan(test['acc_y']/np.sqrt(test['acc_x'] ** 2 + test['acc_z'] ** 2)) * (180/pi)

train['gy_roll'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2))
test['gy_roll'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2))

train['gy_pitch'] = np.arctan(train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2))
test['gy_pitch'] = np.arctan(test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2))

train['gy_math_roll'] = np.arctan(- train['gy_x']/np.sqrt(train['gy_y'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_roll'] = np.arctan(- test['gy_x']/np.sqrt(test['gy_y'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

train['gy_math_pitch'] = np.arctan(train['gy_y']/np.sqrt(train['gy_x'] ** 2 + train['gy_z'] ** 2)) * (180/pi)
test['gy_math_pitch'] = np.arctan(test['gy_y']/np.sqrt(test['gy_x'] ** 2 + test['gy_z'] ** 2)) * (180/pi)

print(train.shape)
train

(20625000, 27)


,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,acc_t,gy_t,acc_mag,gy_mag,acc_vector,gy_vector,acc_YZvector,gy_YZvector,acc_XYvector,gy_XYvector,acc_XZvector,gy_XZvector,gy_Centerofgravity,roll,pitch,math_roll,math_pitch,gy_roll,gy_pitch,gy_math_roll,gy_math_pitch
0,2.307314,-0.029939,-0.432104,0.011776,-0.410859,-0.461007,5.896150,0.765711,5.511308,0.381471,2.347617,0.617633,0.433140,0.617521,2.307508,0.411027,2.347427,0.461158,-0.286697,-0.012753,1.385231,-79.367910,-0.730704,-0.727781,0.019068,-1.092507,-41.698765
1,2.421086,-0.063321,-0.504058,0.026484,-0.528052,-0.365459,6.499033,0.790703,6.119743,0.413101,2.473812,0.642729,0.508020,0.642183,2.421914,0.528716,2.473001,0.366417,-0.289009,-0.025599,1.363966,-78.149478,-1.466742,-0.964174,0.041217,-2.361573,-55.243076
2,2.444664,-0.056749,-0.654199,-0.037962,-0.596041,-0.366767,6.733202,0.869088,6.407580,0.491224,2.531320,0.700874,0.656655,0.699845,2.445323,0.597249,2.530684,0.368727,-0.333590,-0.022421,1.308383,-74.964812,-1.284609,-1.016793,-0.054190,3.104851,-58.257969
3,2.428612,-0.116782,-0.573407,0.066099,-0.725311,-0.401232,6.601997,1.074443,6.240592,0.691433,2.498118,0.831524,0.585178,0.828893,2.431419,0.728317,2.495386,0.406641,-0.353481,-0.046765,1.334352,-76.452715,-2.679451,-1.059819,0.079576,-4.559348,-60.723136
4,2.439475,-0.044220,-0.588886,0.091969,-0.784094,-0.408413,6.655560,1.173732,6.299780,0.790064,2.509936,0.888855,0.590543,0.884085,2.439876,0.789470,2.509547,0.418640,-0.366846,-0.017619,1.333287,-76.391740,-1.009478,-1.080386,0.103655,-5.938975,-61.901532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20624995,-0.840162,0.249148,-0.014579,-0.285672,0.071019,-0.173637,0.827625,0.397891,0.768159,0.116802,0.876447,0.341763,0.249574,0.187599,0.876326,0.294367,0.840288,0.334302,-0.129430,0.288245,-1.282044,73.455690,16.515247,0.209326,-0.989730,56.707354,11.993497
20624996,-0.907620,0.225072,-0.044249,-0.436427,0.091745,-0.087905,0.999536,0.396587,0.876390,0.206613,0.936157,0.454547,0.229381,0.127061,0.935111,0.445966,0.908698,0.445192,-0.144196,0.242800,-1.323252,75.816763,13.911414,0.203234,-1.287489,73.767689,11.644425
20624997,-0.878619,0.186470,-0.029572,-0.336794,0.076047,-0.066560,0.902370,0.283453,0.807616,0.123644,0.898675,0.351630,0.188801,0.101061,0.898188,0.345273,0.879116,0.343308,-0.109103,0.209013,-1.359132,77.872505,11.975573,0.217991,-1.279277,73.297188,12.489986
20624998,-0.818231,0.203014,-0.004935,-0.231834,0.014088,-0.141685,0.739704,0.325725,0.710742,0.074020,0.843055,0.272067,0.203074,0.142384,0.843040,0.232262,0.818246,0.271702,-0.119811,0.243198,-1.327525,76.061567,13.934233,0.051804,-1.020029,58.443343,2.968143


In [17]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [18]:
len_features = train.shape[1] # feature 갯수
X=tf.reshape(np.array(train.iloc[:,0:]),[-1, 600, len_features])
X = X.numpy()
X.shape

(34375, 600, 27)

In [19]:
trainX = X
trainX.shape

(34375, 600, 27)

In [20]:
trainy = to_categorical(y_train_total) 
trainy.shape

(34375, 61)

In [21]:
# fit and evaluate a model
# define model
epochs, batch_size = 30, 128
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]


In [22]:
# reshape data into time steps of sub-sequences
n_steps, n_length = 4, 150
trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))

In [23]:
test_X = tf.reshape(np.array(test.iloc[:,2:]),[-1, 600, len_features])
test_X = test_X.numpy()
test_X = test_X.reshape((test_X.shape[0], n_steps, n_length, n_features))
test_X.shape

(782, 4, 150, 27)

In [24]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(Dropout(0.5)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(n_outputs, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [25]:
# fit network
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size)

Epoch 1/30
269/269 [==============================] - 26s 15ms/step - loss: 2.8270 - accuracy: 0.4817
Epoch 2/30
269/269 [==============================] - 4s 15ms/step - loss: 2.7305 - accuracy: 0.4858
Epoch 3/30
269/269 [==============================] - 5s 19ms/step - loss: 2.7124 - accuracy: 0.4858
Epoch 4/30
269/269 [==============================] - 4s 15ms/step - loss: 2.7185 - accuracy: 0.4858
Epoch 5/30
269/269 [==============================] - 4s 15ms/step - loss: 2.7098 - accuracy: 0.4857
Epoch 6/30
269/269 [==============================] - 4s 15ms/step - loss: 2.7030 - accuracy: 0.4856
Epoch 7/30
269/269 [==============================] - 4s 15ms/step - loss: 2.6951 - accuracy: 0.4858
Epoch 8/30
269/269 [==============================] - 4s 15ms/step - loss: 2.6906 - accuracy: 0.4857
Epoch 9/30
269/269 [==============================] - 4s 15ms/step - loss: 2.6938 - accuracy: 0.4857
Epoch 10/30
269/269 [==============================] - 4s 15ms/step - loss: 2.6798 - accur

In [26]:
prediction=model.predict(test_X)

In [27]:
prediction

array([[0.00444853, 0.00428559, 0.00482745, ..., 0.01104822, 0.00478867,
        0.02086573],
       [0.00533093, 0.00670272, 0.00694733, ..., 0.00777467, 0.00733092,
        0.01614005],
       [0.00576499, 0.01001456, 0.00961671, ..., 0.00518068, 0.01103033,
        0.01215658],
       ...,
       [0.00576499, 0.01001456, 0.00961671, ..., 0.00518068, 0.01103033,
        0.01215658],
       [0.00576499, 0.01001456, 0.00961671, ..., 0.00518068, 0.01103033,
        0.01215658],
       [0.00576499, 0.01001456, 0.00961671, ..., 0.00518068, 0.01103033,
        0.01215658]], dtype=float32)

In [28]:
prediction.shape

(782, 61)

In [29]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3127,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3128,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,3129,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
778,3903,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
779,3904,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
780,3905,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
submission.iloc[:,1:]=prediction

In [31]:
submission

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,3125,0.004449,0.004286,0.004827,0.005718,0.013042,0.005979,0.004943,0.008103,0.028167,0.018501,0.010684,0.013882,0.008289,0.007315,0.013975,0.006919,0.004733,0.006377,0.010264,0.003720,0.006284,0.008319,0.010638,0.007234,0.022866,0.005542,0.458282,0.006504,0.013353,0.004763,0.019466,0.012302,0.007470,0.005846,0.008670,0.005786,0.012437,0.006720,0.008083,0.005712,0.008114,0.004968,0.006521,0.008636,0.006313,0.005537,0.005287,0.005557,0.011211,0.010735,0.013362,0.012156,0.007954,0.005020,0.005414,0.007272,0.007145,0.011644,0.011048,0.004789,0.020866
1,3126,0.005331,0.006703,0.006947,0.007741,0.013296,0.008820,0.006888,0.009551,0.030633,0.013057,0.008336,0.010379,0.005494,0.005105,0.009336,0.008656,0.007283,0.008298,0.014445,0.005332,0.009239,0.010077,0.008526,0.008581,0.014091,0.007744,0.428420,0.009224,0.015963,0.006627,0.013298,0.008316,0.007379,0.008138,0.009517,0.008988,0.012574,0.009390,0.008075,0.007346,0.011240,0.007220,0.008021,0.012476,0.008552,0.007714,0.007290,0.007810,0.009955,0.011291,0.014204,0.010611,0.005672,0.005421,0.007397,0.010926,0.010562,0.009251,0.007775,0.007331,0.016140
2,3127,0.005765,0.010015,0.009617,0.010060,0.012571,0.012266,0.009447,0.011086,0.032287,0.008830,0.005956,0.006945,0.003389,0.003214,0.006043,0.010346,0.010061,0.011027,0.019694,0.007762,0.013075,0.011262,0.006396,0.009383,0.008098,0.010628,0.377679,0.012815,0.018761,0.008926,0.008391,0.005204,0.007097,0.010320,0.009642,0.013016,0.011711,0.013345,0.007373,0.009284,0.014787,0.010053,0.009548,0.017301,0.010881,0.009902,0.009613,0.010383,0.008256,0.011518,0.014981,0.008892,0.003937,0.005321,0.009603,0.015726,0.015372,0.006774,0.005181,0.011030,0.012157
3,3128,0.004449,0.004286,0.004827,0.005718,0.013042,0.005979,0.004943,0.008103,0.028167,0.018501,0.010684,0.013882,0.008289,0.007315,0.013975,0.006919,0.004733,0.006377,0.010264,0.003720,0.006284,0.008319,0.010638,0.007234,0.022866,0.005542,0.458282,0.006504,0.013353,0.004763,0.019466,0.012302,0.007470,0.005846,0.008670,0.005786,0.012437,0.006720,0.008083,0.005712,0.008114,0.004968,0.006521,0.008636,0.006313,0.005537,0.005287,0.005557,0.011211,0.010735,0.013362,0.012156,0.007954,0.005020,0.005414,0.007272,0.007145,0.011644,0.011048,0.004789,0.020866
4,3129,0.005765,0.010015,0.009617,0.010060,0.012571,0.012266,0.009447,0.011086,0.032287,0.008830,0.005956,0.006945,0.003389,0.003214,0.006043,0.010346,0.010061,0.011027,0.019694,0.007762,0.013075,0.011262,0.006396,0.009383,0.008098,0.010628,0.377679,0.012815,0.018761,0.008926,0.008391,0.005204,0.007097,0.010320,0.009642,0.013016,0.011711,0.013345,0.007373,0.009284,0.014787,0.010053,0.009548,0.017301,0.010881,0.009902,0.009613,0.010383,0.008256,0.011518,0.014981,0.008892,0.003937,0.005321,0.009603,0.015726,0.015372,0.006774,0.005181,0.011030,0.012157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,3902,0.005765,0.010015,0.009617,0.010060,0.012571,0.012266,0.009447,0.011086,0.032287,0.008830,0.005956,0.006945,0.003389,0.003214,0.006043,0.010346,0.010061,0.011027,0.019694,0.007762,0.013075,0.011262,0.006396,0.009383,0.008098,0.010628,0.377679,0.012815,0.018761,0.008926,0.008391,0.005204,0.007097,0.010320,0.009642,0.013016,0.011711,0.013345,0.007373,0.009284,0.014787,0.010053,0.009548,0.017301,0.010881,0.009902,0.009613,0.010383,0.008256,0.011518,0.014981,0.008892,0.003937,0.005321,0.009603,0.015726,0.015372,0.006774,0.005181,0.011030,0.012157
778,3903,0.005763,0.010198,0.009789,0.010218,0.012213,0.012468,0.009916,0.011012,0.033607,0.008449,0.005784,0.006668,0.003207,0.003101,0.005869,0.010522,0.010269,0.011617,0.020962,0.008237,0.012969,0.011155,0.006216,0.009405,0.007649

In [33]:
submission.to_csv('cnn_LSTM0(jun).csv', index=False)